In [67]:
import pandas as pd
import glob

In [68]:
csv_files = glob.glob('data/ready to merge/*.csv')

In [69]:
data = pd.concat(
    [pd.read_csv(csv_file).assign(retailer=csv_file.split('/')[-1].split('_')[0]) for csv_file in csv_files],
    ignore_index=True,
)

In [70]:
data.head()

,name,abv,year,size(cL),country,wine_type,price,num_review,rating,url,countrycode,scaled_price,logprice,price_fixed,age,score,retailer
0,Cabalié 2021,13.0,2021,75.0,France,Red Wine,11.99,7475.0,4.5,https://www.laithwaites.co.uk/product/Cabali%C...,FRA,11.9900,1.078819,11.99,1.0,4.5,laithwaites
1,The Black Stump Durif Shiraz 2021,14.5,2021,75.0,Australia,Red Wine,11.99,7445.0,4.5,https://www.laithwaites.co.uk/product/The-Blac...,AUS,11.9900,1.078819,11.99,1.0,4.5,laithwaites
2,Vinha do Fava Reserva 2020,14.5,2020,75.0,Portugal,Red Wine,15.49,544.0,4.5,https://www.laithwaites.co.uk/product/Vinha-do...,PRT,15.4900,1.190051,15.49,2.0,4.5,laithwaites
3,w/o Organic Frappato 2021,13.5,2021,75.0,Italy,Red Wine,11.99,158.0,3.8,https://www.laithwaites.co.uk/product/w/o-Orga...,ITA,11.9900,1.078819,11.99,1.0,3.8,laithwaites
4,Cabalié (1 Litre Wine Box) 2021,13.0,2021,100.0,France,Red Wine,9.99,14.0,4.8,https://www.laithwaites.co.uk/product/Cabali%C...,FRA,7.4925,0.874627,9.99,1.0,4.8,laithwaites


In [71]:
# Fix country names
data['country'] = data['country'].map(
    {
        ': Portugal': 'Portugal',
        'Argentina': 'Argentina',
        'Australia': 'Australia',
        'Austria': 'Austria',
        'Bordeaux': 'France',
        'British': 'UK',
        'Bulgaria': 'Bulgaria',
        'Cahors': 'France',
        'California': 'USA',
        'Canada': 'Canada',
        'Chile': 'Chile',
        'Croatia': 'Croatia',
        'Czech Republic': 'Czech Republic',
        'England': 'UK',
        'France': 'France',
        'Germany': 'Germany',
        'Great Britain': 'UK',
        'Greece': 'Greece',
        'Hungary': 'Hungary',
        'Israel': 'Israel',
        'Italy': 'Italy',
        'Lebanon': 'Lebanon',
        'Marlborough': 'UK',
        'Moldova': 'Moldova',
        'Moldova (Republic Of)': 'Moldova',
        'Multiple Countries': 'Multiple Countries',
        'New Zealand': 'New Zealand',
        'Peru': 'Peru',
        'Portugal': 'Portugal',
        'Produce of the EU': 'EU',
        'Romania': 'Romania',
        'Scotland': 'UK',
        'Slovenia': 'Slovenia',
        'South Africa': 'South Africa',
        'Spain': 'Spain',
        'Turkey': 'Turkey',
        'U': 'USA',
        'USA': 'USA',
        'United Kingdom': 'UK',
        'United States': 'USA',
        'Uruguay': 'Uruguay',
        'Valle Central': 'Chile',
        'Western Australia': 'Australia',
        'Western Cape': 'South Africa',
        'the Island of Madeira': 'Portugal',
        'unknown': 'unknown',
    })

In [72]:
data['year'] = data['year'].replace('Non Vintage', 'NV')

In [73]:
# Normalise wine types
data['wine_type'] = data['wine_type'].map({
    'Dessert': 'Other',
    'Dessert Wine': 'Other',
    'Fortified': 'Other',
    'Low / Alcohol Free Wines': 'Other',
    'Red': 'Red',
    'Red Wine': 'Red',
    'Rose Wine': 'Rose',
    'Rosé': 'Rose',
    'Sherry': 'Other',
    'Sparkling': 'Sparkling',
    'White': 'White',
    'White Wine': 'White',
    'fortified and sherry': 'Other',
    'fortified_and_other': 'Other',
    'orange': 'Other',
    'red': 'Red',
    'rose': 'Rose',
    'sparkling': 'Sparkling',
    'sweet': 'Other',
    'vermouth': 'Other',
    'white': 'White',
})

In [74]:
data.groupby('wine_type').size()

wine_type
Other         581
Red          5423
Rose          393
Sparkling     201
White        2964
dtype: int64

In [75]:
import pycountry

# create a dict f country name and their codes 
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

data["countrycode"] = [countries.get(c, "unknown") for c in data["country"]]

In [76]:
# USA, Maldova are coded as unknown. We might consider removing this column
# Bottles with no ratings are scored 0 in the 'score' column. Pay attention to this or remove this column altogether.

In [77]:
data.to_csv('./data/all_retailers.csv')